In [1]:
import tensorflow as tf

In [2]:
traing_samples_file_path = tf.keras.utils.get_file("trainingSamples.csv", "file:///D:/program/PyRecommendationSystem/DIEN/dataset/sampledata/trainingSamples.csv")

In [3]:
traing_samples_file_path

'C:\\Users\\EDZ\\.keras\\datasets\\trainingSamples.csv'

In [4]:
import pandas as pd
import random
def get_dataset_with_negtive_movie(path, batch_size, seed_num):
    tmp_df = pd.read_csv(path)
    tmp_df.fillna(0, inplace=True)
    random.seed(seed_num)
    negative_movie_df = tmp_df.loc[:, 'userRatedMovie2':'userRatedMovie5'].applymap(lambda x: random.sample(set(range(0, 1001))-set([int(x)]), 1)[0])
    negative_movie_df.columns = ['negtive_userRatedMovie2','negtive_userRatedMovie3','negtive_userRatedMovie4','negtive_userRatedMovie5']
    tmp_df = pd.concat([tmp_df, negative_movie_df], axis=1)
    
    for i in tmp_df.select_dtypes('O').columns:
        print(i)
        tmp_df[i] = tmp_df[i].astype('str')
        
    if tf.__version__ < '2.3.0':
        tmp_df = tmp_df.sample(n=batch_size*(len(tmp_df)//batch_size), random_state=seed_num)
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(tmp_df)))
    dataset = dataset.batch(batch_size)
    return dataset

In [5]:
train_dataset = get_dataset_with_negtive_movie(traing_samples_file_path, 12, seed_num=2020)

movieGenre1
movieGenre2
movieGenre3
userGenre1
userGenre2
userGenre3
userGenre4
userGenre5


In [6]:
for i, data in enumerate(train_dataset):
    if i == 1:
        print(data)
        break

{'movieId': <tf.Tensor: shape=(12,), dtype=int64, numpy=
array([110, 110, 110, 111, 111, 112, 121, 125,  14, 140, 140, 141],
      dtype=int64)>, 'userId': <tf.Tensor: shape=(12,), dtype=int64, numpy=
array([25912,  6613,   675, 24269, 27990, 28675, 18634,  1436,  5325,
       24269, 26005, 14887], dtype=int64)>, 'rating': <tf.Tensor: shape=(12,), dtype=float64, numpy=array([2.5, 4. , 5. , 3. , 5. , 4. , 4. , 4. , 3. , 1. , 3. , 4. ])>, 'timestamp': <tf.Tensor: shape=(12,), dtype=int64, numpy=
array([1111631757,  844549987,  866568538,  856817370, 1197546406,
       1111584204, 1140928282,  840205416,  885753655,  856815351,
        831630191,  830155895], dtype=int64)>, 'label': <tf.Tensor: shape=(12,), dtype=int64, numpy=array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1], dtype=int64)>, 'releaseYear': <tf.Tensor: shape=(12,), dtype=int64, numpy=
array([1995, 1995, 1995, 1976, 1976, 1995, 1992, 1996, 1995, 1996, 1996,
       1996], dtype=int64)>, 'movieGenre1': <tf.Tensor: shape=(12,), dtype=

In [7]:
# Config
RECENT_MOVIES = 5
EMBEDDING_SIZE = 10

# define input for keras model
inputs = {
    'movieAvgRating':  tf.keras.layers.Input(name='movieAvgRating', shape=(), dtype='float32'),
    'movieRatingStddev': tf.keras.layers.Input(name='movieRatingStddev', shape=(), dtype='float32'),
    'movieRatingCount': tf.keras.layers.Input(name='movieRatingCount', shape=(), dtype='int32'),
    'userAvgRating': tf.keras.layers.Input(name='userAvgRating', shape=(), dtype='float32'),
    'userRatingStddev': tf.keras.layers.Input(name='userRatingStddev', shape=(), dtype='float32'),
    'userRatingCount': tf.keras.layers.Input(name='userRatingCount', shape=(), dtype='int32'),
    'releaseYear': tf.keras.layers.Input(name='releaseYear', shape=(), dtype='int32'),
    
    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    'userRatedMovie1': tf.keras.layers.Input(name='userRatedMovie1', shape=(), dtype='int32'),
    'userRatedMovie2': tf.keras.layers.Input(name='userRatedMovie2', shape=(), dtype='int32'),
    'userRatedMovie3': tf.keras.layers.Input(name='userRatedMovie3', shape=(), dtype='int32'),
    'userRatedMovie4': tf.keras.layers.Input(name='userRatedMovie4', shape=(), dtype='int32'),
    'userRatedMovie5': tf.keras.layers.Input(name='userRatedMovie5', shape=(), dtype='int32'),
    
    'userGenre1': tf.keras.layers.Input(name='userGenre1', shape=(), dtype='string'),
    'userGenre2': tf.keras.layers.Input(name='userGenre2', shape=(), dtype='string'),
    'userGenre3': tf.keras.layers.Input(name='userGenre3', shape=(), dtype='string'),
    'userGenre4': tf.keras.layers.Input(name='userGenre4', shape=(), dtype='string'),
    'userGenre5': tf.keras.layers.Input(name='userGenre5', shape=(), dtype='string'),
    'movieGenre1': tf.keras.layers.Input(name='movieGenre1', shape=(), dtype='string'),
    'movieGenre2': tf.keras.layers.Input(name='movieGenre2', shape=(), dtype='string'),
    'movieGenre3': tf.keras.layers.Input(name='movieGenre3', shape=(), dtype='string'),
    
    'negtive_userRatedMovie2': tf.keras.layers.Input(name='negtive_userRatedMovie2', shape=(), dtype='int32'),
    'negtive_userRatedMovie3': tf.keras.layers.Input(name='negtive_userRatedMovie3', shape=(), dtype='int32'),
    'negtive_userRatedMovie4': tf.keras.layers.Input(name='negtive_userRatedMovie4', shape=(), dtype='int32'),
    'negtive_userRatedMovie5': tf.keras.layers.Input(name='negtive_userRatedMovie5', shape=(), dtype='int32'),
    
    'label':tf.keras.layers.Input(name='label', shape=(), dtype='int32')
}

In [8]:
# user id embedding feature
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, EMBEDDING_SIZE)

In [9]:
user_emb_col

EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='userId', number_buckets=30001, default_value=None), dimension=10, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x0000000004BF9198>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True, use_safe_embedding_lookup=True)

In [10]:
# genre feature vocabulary
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller', 'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']

In [11]:
# user genre embedding feature
user_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="userGenre1", vocabulary_list=genre_vocab)
user_genre_embed_col = tf.feature_column.embedding_column(user_genre_col, EMBEDDING_SIZE)

In [12]:
# item genre embedding feature
item_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="movieGenre1", vocabulary_list=genre_vocab)
item_embed_col = tf.feature_column.embedding_column(item_genre_col, EMBEDDING_SIZE)

In [13]:
candidate_movie_col = [tf.feature_column.numeric_column(key='movieId', default_value=0)]

In [14]:
# user behaviors
recent_rate_col = [
    tf.feature_column.numeric_column(key='userRatedMovie1', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie2', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie3', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie4', default_value=0),
    tf.feature_column.numeric_column(key='userRatedMovie5', default_value=0),
]

In [15]:
negative_movie_col = [
    tf.feature_column.numeric_column(key='negtive_userRatedMovie2', default_value=0),
    tf.feature_column.numeric_column(key='negtive_userRatedMovie3', default_value=0),
    tf.feature_column.numeric_column(key='negtive_userRatedMovie4', default_value=0),
    tf.feature_column.numeric_column(key='negtive_userRatedMovie5', default_value=0),
]

In [16]:
# user profile
user_profile = [
    user_emb_col,
    user_genre_embed_col,
    tf.feature_column.numeric_column("userRatingCount"),
    tf.feature_column.numeric_column("userAvgRating"),
    tf.feature_column.numeric_column("userRatingStddev")
]

In [17]:
# context feature
context_feature = [
    item_embed_col,
    tf.feature_column.numeric_column('releaseYear'),
    tf.feature_column.numeric_column('movieRatingCount'),
    tf.feature_column.numeric_column('movieAvgRating'),
    tf.feature_column.numeric_column('movieRatingStddev'),
]

In [18]:
label = [tf.feature_column.numeric_column(key='label', default_value=0),]

In [19]:
candidate_layer = tf.keras.layers.DenseFeatures(candidate_movie_col)(inputs)
user_behaviors_layer = tf.keras.layers.DenseFeatures(recent_rate_col)(inputs)
negative_movie_layer = tf.keras.layers.DenseFeatures(negative_movie_col)(inputs)
user_profile_layer = tf.keras.layers.DenseFeatures(user_profile)(inputs)
context_feature_layer = tf.keras.layers.DenseFeatures(context_feature)(inputs)
y_true = tf.keras.layers.DenseFeatures(label)(inputs)

In [20]:
# Activate Unit
movie_emb_layer = tf.keras.layers.Embedding(input_dim=1001, output_dim=EMBEDDING_SIZE, mask_zero=True)

In [21]:
user_behaviors_emb_layer = movie_emb_layer(user_behaviors_layer)
candidate_emb_layer = movie_emb_layer(candidate_layer)
negative_movie_emd_layer = movie_emb_layer(negative_movie_layer)

In [ ]:
class attentiom(tf.keras.layers.Layer):
    
    def __init__(self, embedding_size=EMBEDDING_SIZE, time_length=5, ):
        super().__init__()
        self.time_length = time_length
        self.embedding_size = embedding_size
        self.RepeatVector_time = tf.keras.layers.RepeatVector(self.time_length)
        self.RepeatVector_emb = tf.keras.layers.RepeatVector(self.embedding_size)
        self.Muitiply = tf.keras.layers.Multiply()
        self.Dense32 = tf.keras.layers.Dense(32, activation='sigmoid')
        self.Dense1 = tf.keras.layers.Dense(1, activation='sigmoid')
        self.Flatten = tf.keras.layers.Flatten()
        self.Permute = tf.keras